<a href="https://colab.research.google.com/github/kazeo48/MAchine-Learning/blob/main/Preprocessing_LGBT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Preprocessing Data Sentimen LGBT

In [2]:
# Import Library
import pandas as pd
import numpy as np
import re
import nltk
import string
import matplotlib.pyplot as plt
!pip install emoji
import emoji

!pip install snscrape
import snscrape.modules.twitter as sntwitter
import tweepy
import csv

from tweepy import OAuthHandler
import string
#NLTK
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
!pip install sastrawi
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory

nltk.download('stopwords')
! pip install openpyxl

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 240.9/240.9 KB 6.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for emoji: filename=emoji-2.2.0-py3-none-any.whl size=234926 sha256=e41f6d66e9d5252db1e3a4a5d4730aad437c6dd565a37130f4214427707afb7d
  Stored in directory: /root/.cache/pip/wheels/86/62/9e/a6b27a681abcde69970dbc0326ff51955f3beac72f15696984
Successfully built emoji
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.1/59.1 KB 2.1 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.7/209.7 KB 5.5 MB/s eta 0:00:00


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
query = "lgbt lang:id until:2022-08-31 since:2022-08-01 -filter:links -filter:replies"
tweets = []
limit = 1000000


for tweet in sntwitter.TwitterSearchScraper(query).get_items():
    
    # print(vars(tweet))
    # break
    if len(tweets) == limit:
        break
    else:
        tweets.append([tweet.date, tweet.username, tweet.content])
        
df = pd.DataFrame(tweets, columns=['Date', 'User', 'Tweet'])
print(df)

# to save to csv
df.to_csv('tweetlgbt.csv')

<ipython-input-3-b4ab93f5615f>:13: FutureWarning: username is deprecated, use user.username instead
  tweets.append([tweet.date, tweet.username, tweet.content])


                          Date           User  \
0    2022-08-30 23:35:11+00:00      ripwest__   
1    2022-08-30 22:43:15+00:00   AgathaAnne10   
2    2022-08-30 22:29:39+00:00    domitiepsny   
3    2022-08-30 22:12:22+00:00      taemstars   
4    2022-08-30 21:53:11+00:00        basaheh   
...                        ...            ...   
2215 2022-08-01 00:54:08+00:00   shinobukocok   
2216 2022-08-01 00:48:36+00:00       pisalute   
2217 2022-08-01 00:35:52+00:00  goodbyeicarus   
2218 2022-08-01 00:35:22+00:00      Qwetoprak   
2219 2022-08-01 00:30:38+00:00     DEFence_hr   

                                                  Tweet  
0     pelangi itu bagus, tapi karna para lgbt mengha...  
1                                                  Lgbt  
2                                             lgbt /neg  
3               mn o key eh mto viadinho twink poc lgbt  
4     meta juga melindungi komunitas LGBT. Jika ada ...  
...                                                 ...  
2215 

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
# Import data
data = pd.read_excel('/content/drive/MyDrive/Tweet.xlsx', usecols=['Tweet'] )
data

,Tweet
0,"sekarang lagi ngetrend lgbt..,, yah maaf ni cu..."
1,meta juga melindungi komunitas LGBT. Jika ada ...
2,Tidak boleh bilang LGBT adalah sebuah penyakit...
3,"Negara ni banyak bala bukan sebab lgbt, rasis ..."
4,orang2222 bersih di negri kita hampir semuanya...
...,...
995,salah satu hal yang paling bikin gw takut adal...
996,"cw lgbt ch..., buat sticker nsfw lgbt di tele ..."
997,ngatain lgbt tapi lu pacaran sampe hs lawak am...
998,orang yang sok open minded dan membolehkan pas...


### Langkah-Langkah Preprocessing Data
### 1. Data Cleaning (Remove Regex)
Tujuan dari Cleaning ini adalah untuk Menghapus tanda baca, url, hashtagh,karakter, angka dan lain-lain

In [ ]:
# Proses Cleansing Data
def remove(Text):
    # remove stock market tickers like $GE
    Text = re.sub(r'\$\w*', '',str(Text))
    # replace kata yang berulang-ulang ('oooooo' menjadi '00')
    Text = re.sub(r'(.)\1+', r'\1\1', Text)
    # Replace 2+ dots with space
    Text = re.sub(r'\.{2,}', ' ', Text)
    # replace emoji dengan text
    Text = emoji.demojize(Text)
    #remove @username
    Text = re.sub('@[^\s]+','',Text)
    # remove old style retweet text "RT"
    Text = re.sub(r'^RT[\s]+', '', Text)
    #remove angka
    Text = re.sub('[0-9]+', '', Text)
    #remove url
    Text = re.sub(r"http\S+", "", Text)
    # remove hashtags
    Text = re.sub(r'#', '', Text)
    # Strip space, " and ' from tweet
    Text = Text.strip(' "\'')
    # Replace multiple spaces with a single space
    Text = re.sub(r'\s+', ' ', Text)
    #hapus tanda baca
    Text = Text.translate(str.maketrans("","",string.punctuation))
    #hapus karakter
    Text = Text.replace("\n",' ')

    return Text

data['clean'] = data['Tweet'].apply(lambda x: remove(x))

data

,Tweet,clean
0,"sekarang lagi ngetrend lgbt..,, yah maaf ni cu...",sekarang lagi ngetrend lgbt yah maaf ni cuma ...
1,meta juga melindungi komunitas LGBT. Jika ada ...,meta juga melindungi komunitas LGBT Jika ada y...
2,Tidak boleh bilang LGBT adalah sebuah penyakit...,Tidak boleh bilang LGBT adalah sebuah penyakit...
3,"Negara ni banyak bala bukan sebab lgbt, rasis ...",Negara ni banyak bala bukan sebab lgbt rasis d...
4,orang2222 bersih di negri kita hampir semuanya...,orang bersih di negri kita hampir semuanya saa...
...,...,...
995,salah satu hal yang paling bikin gw takut adal...,salah satu hal yang paling bikin gw takut adal...
996,"cw lgbt ch..., buat sticker nsfw lgbt di tele ...",cw lgbt ch buat sticker nsfw lgbt di tele ap...
997,ngatain lgbt tapi lu pacaran sampe hs lawak am...,ngatain lgbt tapi lu pacaran sampe hs lawak am...
998,orang yang sok open minded dan membolehkan pas...,orang yang sok open minded dan membolehkan pas...


### 2. Case Folding
Tujuan dari case folding untuk mengubah semua huruf dalam dokumen menjadi huruf kecil. Hanya huruf ‘a’ sampai ‘z’ yang diterima. 

In [ ]:
# proses case folding 
data['case_folding'] = data['clean'].str.lower()
data

,Tweet,clean,case_folding
0,"sekarang lagi ngetrend lgbt..,, yah maaf ni cu...",sekarang lagi ngetrend lgbt yah maaf ni cuma ...,sekarang lagi ngetrend lgbt yah maaf ni cuma ...
1,meta juga melindungi komunitas LGBT. Jika ada ...,meta juga melindungi komunitas LGBT Jika ada y...,meta juga melindungi komunitas lgbt jika ada y...
2,Tidak boleh bilang LGBT adalah sebuah penyakit...,Tidak boleh bilang LGBT adalah sebuah penyakit...,tidak boleh bilang lgbt adalah sebuah penyakit...
3,"Negara ni banyak bala bukan sebab lgbt, rasis ...",Negara ni banyak bala bukan sebab lgbt rasis d...,negara ni banyak bala bukan sebab lgbt rasis d...
4,orang2222 bersih di negri kita hampir semuanya...,orang bersih di negri kita hampir semuanya saa...,orang bersih di negri kita hampir semuanya saa...
...,...,...,...
995,salah satu hal yang paling bikin gw takut adal...,salah satu hal yang paling bikin gw takut adal...,salah satu hal yang paling bikin gw takut adal...
996,"cw lgbt ch..., buat sticker nsfw lgbt di tele ...",cw lgbt ch buat sticker nsfw lgbt di tele ap...,cw lgbt ch buat sticker nsfw lgbt di tele ap...
997,ngatain lgbt tapi lu pacaran sampe hs lawak am...,ngatain lgbt tapi lu pacaran sampe hs lawak am...,ngatain lgbt tapi lu pacaran sampe hs lawak am...
998,orang yang sok open minded dan membolehkan pas...,orang yang sok open minded dan membolehkan pas...,orang yang sok open minded dan membolehkan pas...


# 3. Tokenizing
Tokenizing adalah proses pemisahan teks menjadi potongan-potongan yang disebut sebagai token

In [ ]:
# proses tokenizing, proses pemisahan kata
from nltk.tokenize import word_tokenize
import nltk
nltk.download('punkt')
#NLTK word tokenize
def word_tokenize_wrapper(Tweets):
 return word_tokenize(Tweets)
data['Tokenizing'] = data['case_folding'].apply(word_tokenize_wrapper)
data

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


,Tweet,clean,case_folding,Tokenizing
0,"sekarang lagi ngetrend lgbt..,, yah maaf ni cu...",sekarang lagi ngetrend lgbt yah maaf ni cuma ...,sekarang lagi ngetrend lgbt yah maaf ni cuma ...,"[sekarang, lagi, ngetrend, lgbt, yah, maaf, ni..."
1,meta juga melindungi komunitas LGBT. Jika ada ...,meta juga melindungi komunitas LGBT Jika ada y...,meta juga melindungi komunitas lgbt jika ada y...,"[meta, juga, melindungi, komunitas, lgbt, jika..."
2,Tidak boleh bilang LGBT adalah sebuah penyakit...,Tidak boleh bilang LGBT adalah sebuah penyakit...,tidak boleh bilang lgbt adalah sebuah penyakit...,"[tidak, boleh, bilang, lgbt, adalah, sebuah, p..."
3,"Negara ni banyak bala bukan sebab lgbt, rasis ...",Negara ni banyak bala bukan sebab lgbt rasis d...,negara ni banyak bala bukan sebab lgbt rasis d...,"[negara, ni, banyak, bala, bukan, sebab, lgbt,..."
4,orang2222 bersih di negri kita hampir semuanya...,orang bersih di negri kita hampir semuanya saa...,orang bersih di negri kita hampir semuanya saa...,"[orang, bersih, di, negri, kita, hampir, semua..."
...,...,...,...,...
995,salah satu hal yang paling bikin gw takut adal...,salah satu hal yang paling bikin gw takut adal...,salah satu hal yang paling bikin gw takut adal...,"[salah, satu, hal, yang, paling, bikin, gw, ta..."
996,"cw lgbt ch..., buat sticker nsfw lgbt di tele ...",cw lgbt ch buat sticker nsfw lgbt di tele ap...,cw lgbt ch buat sticker nsfw lgbt di tele ap...,"[cw, lgbt, ch, buat, sticker, nsfw, lgbt, di, ..."
997,ngatain lgbt tapi lu pacaran sampe hs lawak am...,ngatain lgbt tapi lu pacaran sampe hs lawak am...,ngatain lgbt tapi lu pacaran sampe hs lawak am...,"[ngatain, lgbt, tapi, lu, pacaran, sampe, hs, ..."
998,orang yang sok open minded dan membolehkan pas...,orang yang sok open minded dan membolehkan pas...,orang yang sok open minded dan membolehkan pas...,"[orang, yang, sok, open, minded, dan, memboleh..."


In [ ]:
# proses tokenizing, proses pemisahan kata
from nltk.tokenize import word_tokenize
import nltk
nltk.download('punkt')
#NLTK word tokenize
def word_tokenize_wrapper(Tweets):
 return word_tokenize(Tweets)
data['Tokenizing'] = data['case_folding'].apply(word_tokenize_wrapper)
data

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


,Tweet,clean,case_folding,Tokenizing
0,"sekarang lagi ngetrend lgbt..,, yah maaf ni cu...",sekarang lagi ngetrend lgbt yah maaf ni cuma ...,sekarang lagi ngetrend lgbt yah maaf ni cuma ...,"[sekarang, lagi, ngetrend, lgbt, yah, maaf, ni..."
1,meta juga melindungi komunitas LGBT. Jika ada ...,meta juga melindungi komunitas LGBT Jika ada y...,meta juga melindungi komunitas lgbt jika ada y...,"[meta, juga, melindungi, komunitas, lgbt, jika..."
2,Tidak boleh bilang LGBT adalah sebuah penyakit...,Tidak boleh bilang LGBT adalah sebuah penyakit...,tidak boleh bilang lgbt adalah sebuah penyakit...,"[tidak, boleh, bilang, lgbt, adalah, sebuah, p..."
3,"Negara ni banyak bala bukan sebab lgbt, rasis ...",Negara ni banyak bala bukan sebab lgbt rasis d...,negara ni banyak bala bukan sebab lgbt rasis d...,"[negara, ni, banyak, bala, bukan, sebab, lgbt,..."
4,orang2222 bersih di negri kita hampir semuanya...,orang bersih di negri kita hampir semuanya saa...,orang bersih di negri kita hampir semuanya saa...,"[orang, bersih, di, negri, kita, hampir, semua..."
...,...,...,...,...
995,salah satu hal yang paling bikin gw takut adal...,salah satu hal yang paling bikin gw takut adal...,salah satu hal yang paling bikin gw takut adal...,"[salah, satu, hal, yang, paling, bikin, gw, ta..."
996,"cw lgbt ch..., buat sticker nsfw lgbt di tele ...",cw lgbt ch buat sticker nsfw lgbt di tele ap...,cw lgbt ch buat sticker nsfw lgbt di tele ap...,"[cw, lgbt, ch, buat, sticker, nsfw, lgbt, di, ..."
997,ngatain lgbt tapi lu pacaran sampe hs lawak am...,ngatain lgbt tapi lu pacaran sampe hs lawak am...,ngatain lgbt tapi lu pacaran sampe hs lawak am...,"[ngatain, lgbt, tapi, lu, pacaran, sampe, hs, ..."
998,orang yang sok open minded dan membolehkan pas...,orang yang sok open minded dan membolehkan pas...,orang yang sok open minded dan membolehkan pas...,"[orang, yang, sok, open, minded, dan, memboleh..."


Filtering

### 4. Stemming
Stemming adalah proses menghilangkan infleksi kata ke bentuk dasarnya, namun bentuk dasar tersebut tidak berarti sama dengan akar kata (root word). Misalnya kata “mendengarkan”, “dengarkan”, “didengarkan” akan ditransformasi menjadi kata “dengar

In [ ]:
# proses stemming
#stemming harus kata yng telah ditoken agar stemmingnya rapi
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory

def stemming(Tweets):
    factory = StemmerFactory()
    stemmer = factory.create_stemmer()
    do = []
    for w in Tweets:
        dt = stemmer.stem(w)
        do.append(dt)
    d_clean=[]
    d_clean=" ".join(do)
    print(d_clean)
    return d_clean
data['Stemming'] = data['Tokenizing'].apply(stemming)
data

sekarang lagi ngetrend lgbt yah maaf ni cuma mau ngingetin kalian ke kaum sodom masih inget kan kisah nya nabi luth jangan lupa di tonton kisah nabi luth sama kaum sodom
meta juga lindung komunitas lgbt jika ada yang berani posting lawan lgbt bakal hapus dan akun anda bisa kunci
tidak boleh bilang lgbt adalah buah sakit tapi bilang homophobia itu sakit thoughts
negara ni banyak bala bukan sebab lgbt ras dan angkat dengan tapi sbb banyak manusia bangang kat sini buat menara konek kat kl
orang bersih di negri kita hampir semua saat ini sedang jadi lgbt lumpuh gagu buta amp tuli sediih lihat
begini strata lgbt di negri ini lesbian dan gay sangat musuh krn tentang dgn agama bisexual msh aman bisa nikah transgender paling aman bisa wawancara tv berapa operasi dan apa aja hebat kan blokk
sampah betul geng geng lgbt dok obsess dgn pelangi asal warna pelangi dok habaq kita support lgbt harey betul
saya dukung nyata wagub jabar soal nikah muda biar ga kena aids kalo bom siap jangan pacar kalo p

,Tweet,clean,case_folding,Tokenizing,Stemming
0,"sekarang lagi ngetrend lgbt..,, yah maaf ni cu...",sekarang lagi ngetrend lgbt yah maaf ni cuma ...,sekarang lagi ngetrend lgbt yah maaf ni cuma ...,"[sekarang, lagi, ngetrend, lgbt, yah, maaf, ni...",sekarang lagi ngetrend lgbt yah maaf ni cuma m...
1,meta juga melindungi komunitas LGBT. Jika ada ...,meta juga melindungi komunitas LGBT Jika ada y...,meta juga melindungi komunitas lgbt jika ada y...,"[meta, juga, melindungi, komunitas, lgbt, jika...",meta juga lindung komunitas lgbt jika ada yang...
2,Tidak boleh bilang LGBT adalah sebuah penyakit...,Tidak boleh bilang LGBT adalah sebuah penyakit...,tidak boleh bilang lgbt adalah sebuah penyakit...,"[tidak, boleh, bilang, lgbt, adalah, sebuah, p...",tidak boleh bilang lgbt adalah buah sakit tapi...
3,"Negara ni banyak bala bukan sebab lgbt, rasis ...",Negara ni banyak bala bukan sebab lgbt rasis d...,negara ni banyak bala bukan sebab lgbt rasis d...,"[negara, ni, banyak, bala, bukan, sebab, lgbt,...",negara ni banyak bala bukan sebab lgbt ras dan...
4,orang2222 bersih di negri kita hampir semuanya...,orang bersih di negri kita hampir semuanya saa...,orang bersih di negri kita hampir semuanya saa...,"[orang, bersih, di, negri, kita, hampir, semua...",orang bersih di negri kita hampir semua saat i...
...,...,...,...,...,...
995,salah satu hal yang paling bikin gw takut adal...,salah satu hal yang paling bikin gw takut adal...,salah satu hal yang paling bikin gw takut adal...,"[salah, satu, hal, yang, paling, bikin, gw, ta...",salah satu hal yang paling bikin gw takut adal...
996,"cw lgbt ch..., buat sticker nsfw lgbt di tele ...",cw lgbt ch buat sticker nsfw lgbt di tele ap...,cw lgbt ch buat sticker nsfw lgbt di tele ap...,"[cw, lgbt, ch, buat, sticker, nsfw, lgbt, di, ...",cw lgbt ch buat sticker nsfw lgbt di tele apa ...
997,ngatain lgbt tapi lu pacaran sampe hs lawak am...,ngatain lgbt tapi lu pacaran sampe hs lawak am...,ngatain lgbt tapi lu pacaran sampe hs lawak am...,"[ngatain, lgbt, tapi, lu, pacaran, sampe, hs, ...",ngatain lgbt tapi lu pacar sampe hs lawak amat...
998,orang yang sok open minded dan membolehkan pas...,orang yang sok open minded dan membolehkan pas...,orang yang sok open minded dan membolehkan pas...,"[orang, yang, sok, open, minded, dan, memboleh...",orang yang sok open minded dan boleh pasang lg...


In [ ]:
from nltk.tokenize import word_tokenize
import nltk
nltk.download('punkt')
#NLTK word tokenize
def word_tokenize_wrapper(Tweets):
 return word_tokenize(Tweets)
data['Retokenizing'] = data['Stemming'].apply(word_tokenize_wrapper)
data

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


,Tweet,clean,case_folding,Tokenizing,Stemming,Retokenizing
0,"sekarang lagi ngetrend lgbt..,, yah maaf ni cu...",sekarang lagi ngetrend lgbt yah maaf ni cuma ...,sekarang lagi ngetrend lgbt yah maaf ni cuma ...,"[sekarang, lagi, ngetrend, lgbt, yah, maaf, ni...",sekarang lagi ngetrend lgbt yah maaf ni cuma m...,"[sekarang, lagi, ngetrend, lgbt, yah, maaf, ni..."
1,meta juga melindungi komunitas LGBT. Jika ada ...,meta juga melindungi komunitas LGBT Jika ada y...,meta juga melindungi komunitas lgbt jika ada y...,"[meta, juga, melindungi, komunitas, lgbt, jika...",meta juga lindung komunitas lgbt jika ada yang...,"[meta, juga, lindung, komunitas, lgbt, jika, a..."
2,Tidak boleh bilang LGBT adalah sebuah penyakit...,Tidak boleh bilang LGBT adalah sebuah penyakit...,tidak boleh bilang lgbt adalah sebuah penyakit...,"[tidak, boleh, bilang, lgbt, adalah, sebuah, p...",tidak boleh bilang lgbt adalah buah sakit tapi...,"[tidak, boleh, bilang, lgbt, adalah, buah, sak..."
3,"Negara ni banyak bala bukan sebab lgbt, rasis ...",Negara ni banyak bala bukan sebab lgbt rasis d...,negara ni banyak bala bukan sebab lgbt rasis d...,"[negara, ni, banyak, bala, bukan, sebab, lgbt,...",negara ni banyak bala bukan sebab lgbt ras dan...,"[negara, ni, banyak, bala, bukan, sebab, lgbt,..."
4,orang2222 bersih di negri kita hampir semuanya...,orang bersih di negri kita hampir semuanya saa...,orang bersih di negri kita hampir semuanya saa...,"[orang, bersih, di, negri, kita, hampir, semua...",orang bersih di negri kita hampir semua saat i...,"[orang, bersih, di, negri, kita, hampir, semua..."
...,...,...,...,...,...,...
995,salah satu hal yang paling bikin gw takut adal...,salah satu hal yang paling bikin gw takut adal...,salah satu hal yang paling bikin gw takut adal...,"[salah, satu, hal, yang, paling, bikin, gw, ta...",salah satu hal yang paling bikin gw takut adal...,"[salah, satu, hal, yang, paling, bikin, gw, ta..."
996,"cw lgbt ch..., buat sticker nsfw lgbt di tele ...",cw lgbt ch buat sticker nsfw lgbt di tele ap...,cw lgbt ch buat sticker nsfw lgbt di tele ap...,"[cw, lgbt, ch, buat, sticker, nsfw, lgbt, di, ...",cw lgbt ch buat sticker nsfw lgbt di tele apa ...,"[cw, lgbt, ch, buat, sticker, nsfw, lgbt, di, ..."
997,ngatain lgbt tapi lu pacaran sampe hs lawak am...,ngatain lgbt tapi lu pacaran sampe hs lawak am...,ngatain lgbt tapi lu pacaran sampe hs lawak am...,"[ngatain, lgbt, tapi, lu, pacaran, sampe, hs, ...",ngatain lgbt tapi lu pacar sampe hs lawak amat...,"[ngatain, lgbt, tapi, lu, pacar, sampe, hs, la..."
998,orang yang sok open minded dan membolehkan pas...,orang yang sok open minded dan membolehkan pas...,orang yang sok open minded dan membolehkan pas...,"[orang, yang, sok, open, minded, dan, memboleh...",orang yang sok open minded dan boleh pasang lg...,"[orang, yang, sok, open, minded, dan, boleh, p..."


In [ ]:
6# Simpan Data
data.to_excel("preprocessing_lgbt.xlsx")